## Training
This command will train a 3D U-net network based on users' input then, after succesful training, a model file will be stored in the result folder which serves for the MWR prediction. 
For detail instruction, please run the following command

In [2]:
%%bash
mwr_cli.py train -h

INFO: Showing help with the command 'mwr_cli.py train -- --help'.

NAME
    mwr_cli.py train - Preprocess tomogram and train u-net model on generated subtomos

SYNOPSIS
    mwr_cli.py train INPUT_DIR <flags>

DESCRIPTION
    Preprocess tomogram and train u-net model on generated subtomos

POSITIONAL ARGUMENTS
    INPUT_DIR
        path to tomogram from which subtomos are sampled; format: .mwr or .rec

FLAGS
    --gpuID=GPUID
        (0,1,2,3) The gpuID to used during the training. e.g 0,1,2,3.
    --mask=MASK
        (None) if sample subtomos with a mask to exclude background region. (None) the path to mask file or None
    --noise_folder=NOISE_FOLDER
        Add noise during training, Set None to disable noise reduction.
    --iterations=ITERATIONS
        (50) Number of training iterations.
    --datas_are_subtomos=DATAS_ARE_SUBTOMOS
        (False) Is your trainin data subtomograms?
    --subtomo_dir=SUBTOMO_DIR
        (subtomo) The folder where the input subtomograms at.
    --dat

## Parameters tuning
The following parameters need to be tuned with respect to user's data and hardware.

iteration parameter defines the number of iterations that will be used to correct missing wedge. In each iteration, three steps will be performed: preprocessing, training, and prediction.
 
In preprocessing, the sub-tomograms with size **cube_size** are extracted randomly from the tomograms. If masks are applied, the center of the sub-tomograms should be within the masked areas. Those sub-tomograms are then rotated and cropped to the size serve as the input. Those same sub-tomograms are rotated, added missing wedge and then cropped to the same size to serve as the output of the network. Usually the **crop_size** is 1.5 time larger than the **cube_size**. By default, each sub-tomograms are rotated 16 times making 16 data pairs. Those generated input-output data pairs store in the **data** folder and will be used in the following training. The preprocessing step can only be performed with cpu, which specified in the parameter **preprocessing_ncpus**.
 
The training step used GPU to train the neural network. The ID of GPU is can be assigned in **gpuID**. The training step is divided into several **epochs**. Each epoch will traverse through the data set. The data pairs are grouped into batches to feed into each epoch. It is recommended that the **batchsize** times the **step_per_epoch** approximately equals the number of the data pairs. If you are using multiple GPUs, the **batchsize** should be divisible to the number of GPU. The trained neural networks are saved in the **result_dir** folder named model_iterxx.h5. 
 
The prediction step performs the missing wedge correction of each sub-tomograms. Similar to the training, the sub-tomograms are grouped in to batches **predict_batch_size**.  The predicted sub-tomograms will be used for processing for the subsequent iterations.
 
## that’s it
The resulting **model_iterxx.h5** is the network to be used in the next module: Missing wedge correction of the entire tomogram.